In [0]:
import time
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession 
from pyspark.sql.functions import *
from pyspark.sql.types import ArrayType,IntegerType,MapType

In [0]:
#à vérifier mais probablement inutile
from pyspark.sql.functions import split

In [0]:
#attention fonctionnement différent de RDD / iic la fonction prend en argument la colonne spécifiée et non (key,value)
#il faut passer Key en paramètre
import time
#@udf(MapType(IntegerType(),IntegerType()))
@udf(ArrayType(ArrayType(IntegerType())))
def DFreducerBis(key,value):
    min=key
    valuesList=[]
    listoutput=[]
#   global accu
    for i in value:
        if i <min: 
            min=i
        valuesList.append(i)
    if min < key:
        listoutput.append((0,key,min))
        for j in valuesList:
            if min != j:
              listoutput.append((1,j,min))
#             accu.add(1)
#   print(listoutput)
    return listoutput

In [0]:
#2ieme méthode de comptage du nombre de paires créées par le reduce
#cette méthode utilise le même code que la version RDD
#attention fonctionnement différent de RDD / iic la fonction prend en argument la colonne spécifiée et non (key,value)
#il faut passer Key en paramètre
from pyspark.sql.types import ArrayType,IntegerType,MapType
#@udf(MapType(IntegerType(),IntegerType()))
@udf(ArrayType(ArrayType(StringType())))
def DFreducer(key,value):
    min=key
    valuesList=[]
    listoutput=[]
#    global accu
    for i in value:
        if i <min: 
            min=i
        valuesList.append(i)
    if min < key:
        listoutput.append((key,min))
        for j in valuesList:
            if min != j:
              listoutput.append((j,min))
#              accu.add(1)
#   print(listoutput)
    return listoutput

In [0]:
#fichier web_Google.txt qui présente une difficulté de préparation des données :lignes à supprimer + séparateur /p
#methode 1 en passant par un RDD (plus facile à manipuler) : solution trouvée sur internet
path = "/FileStore/tables/RDD/" 
rdd_Prep = sc.textFile(path + "/web_Google.txt", use_unicode="False")

rdd_Init=rdd_Prep.filter(lambda x: "#" not in x)\
               .map(lambda x : x.split("\t"))\
               .map(lambda x : (int(x[0]), int(x[1])))
DF_Init=spark.createDataFrame(rdd_Init)

DF=DF_Init.withColumn("_1",DF_Init["_1"].cast("int")).withColumnRenamed("_1","key")\
                     .withColumn("_2",DF_Init["_2"].cast("int")).withColumnRenamed("_2","value")



In [0]:
#fichier web_Google.txt qui présente une difficulté de préparation des données :lignes à supprimer + séparateur /p
#methode 1 en passant par un RDD (plus facile à manipuler) : solution trouvée sur internet
path = "/FileStore/tables/RDD/" 
DF_Prep=spark.read.csv(path + "/web_Google.txt")
DF_Init=DF_Prep.where("_c0 != '# Directed graph (each unordered pair of nodes is saved once): web-Google.txt '")\
    .where(("_c0 != '# Webgraph from the Google programming contest'"))\
    .where(("_c0 != '# Nodes: 875713 Edges: 5105039'"))\
    .where(("_c0 != '# FromNodeId\tToNodeId'"))

DF = DF_Init.withColumn('key', split(DF_Init['_c0'], '\t').getItem(0).cast("int")) \
       .withColumn('value', split(DF_Init['_c0'], '\t').getItem(1).cast("int")).drop(col('_c0'))

In [0]:
path = "/FileStore/tables/RDD/" 
DF_Init=spark.read.csv(path + "/facebook_combined.txt",sep=" ")

In [0]:
path = "/FileStore/tables/RDD/" 
DF_Prep=spark.read.csv(path + "/HR_edges.csv",sep=",")
DF_Init=DF_Prep.where("_c0 != 'node_1'")

In [0]:
path = "/FileStore/tables/RDD/" 
DF_Prep=spark.read.csv(path + "/large_twitch_edges.csv",sep=",")
DF_Init=DF_Prep.where("_c0 != 'numeric_id_1'")

In [0]:
path = "/FileStore/tables/RDD/" 
DF_Prep=spark.read.csv(path + "/artist_edges.csv",sep=",")
DF_Init=DF_Prep.where("_c0 != 'node_1'")

In [0]:
path = "/FileStore/tables/RDD/" 
DF_Init=spark.read.csv(path + "/soc_pokec_relationships.txt",sep="\t")

In [0]:
#programme complet avec le reducer bis (astuce de créer des triplets avec 0 ou 1 en fct de la création d'une pair ou non)
#ligne de commande à supprimer dans le cas de web-google avec modification en DF
#DF=DF_Init.withColumn("_c0",DF_Init["_c0"].cast("int")).withColumnRenamed("_c0","key")\
#                     .withColumn("_c1",DF_Init["_c1"].cast("int")).withColumnRenamed("_c1","value")

#Initialisation de la boucle
boucle = True
t0=time.time()

while boucle == True:

  #map du CFF
  MapDF=DF.union(DF.select('value','key')).groupBy("key").agg(collect_list("value").alias("MapOutput"))

  #fonction reduce du CCF Iterate RDD fichier de chiffres
  ReduceDF=MapDF.withColumn("ListnewPairs",DFreducerBis("key","MapOutput")).select(explode(col('ListnewPairs'))).select([col("col")[i] for i in range(3)])

  compteur=ReduceDF.where("col[0]=1").count()
  print(compteur)

  ##suite #fonction Reduce de CFF Dedup sur base RDD liste de chiffres pour vérifier la suppression des doublons de tuples
  DF=ReduceDF.withColumnRenamed("col[1]","key").withColumnRenamed("col[2]","value").select("key","value").distinct()

  if compteur == 0:
    print("arret")
    boucle = False

t1=time.time()
#le round de python interfère l'opérateur round de pspark sql (qui s'applique à des colonnes)
print("c'est la fin; durée totale de :",t1-t0)

8552232
4758451
3278772
3888454
1905323
86783
1318
0
arret
c'est la fin; durée totale de : 1149.521509885788


In [0]:
#autre méthode de calcul du compteur

In [0]:
#essai programme avec l'autre méthode de calcul du nbre de paires créées par le reduce
#voir les annexes pour les codes de vérification de fonctionnement
DF=DF_Init.withColumn("_c0",DF_Init["_c0"].cast("int")).withColumnRenamed("_c0","key")\
                     .withColumn("_c1",DF_Init["_c1"].cast("int")).withColumnRenamed("_c1","value")

#Initialisation de la boucle
boucle = True
t0=time.time()

while boucle == True:

  #map du CFF
  MapDF=DF.union(DF.select('value','key')).groupBy("key").agg(collect_list("value").alias("MapOutput"))

  #fonction reduce du CCF Iterate DF fichier de chiffres
  ReduceDF_comptage=MapDF.withColumn("ListnewPairs",DFreducer("key","MapOutput"))
  ReduceDF=ReduceDF_comptage.select(explode(col('ListnewPairs'))).select([col("col")[i] for i in range(2)])

  compteur=ReduceDF_comptage.withColumn("minimum_array",array_min("MapOutput"))\
    .filter(col('key') > col('minimum_array')).withColumn("compteur",size("MapOutput")-1).agg(sum(col('compteur'))).collect()[0][0] 
  print(compteur)

  ##suite #fonction Reduce de CFF Dedup sur base RDD liste de chiffres pour vérifier la su#ppression des doublons de tuples
  DF=ReduceDF.withColumnRenamed("col[0]","key").withColumnRenamed("col[1]","value").select("key","value").distinct()

  if compteur == 0:
    print("arret")
    boucle = False

t1=time.time()
#le round de python interfère l'opérateur round de pspark sql (qui s'applique à des colonnes)
print("c'est la fin; durée totale de :",t1-t0)

171914
18733
6516
1658
0
arret
c'est la fin; durée totale de : 38.1501681804657


In [0]:
# annexes / divers codes pour essais et contrôles

In [0]:
#Initialisation de la boucle / save du cas google - àmettre en annexe
boucle = True
t0=time.time()

while boucle == True:

  #map du CFF
  MapDF=DF.union(DF.select('value','key')).groupBy("key").agg(collect_list("value").alias("MapOutput"))

  #fonction reduce du CCF Iterate RDD fichier de chiffres
  ReduceDF=MapDF.withColumn("ListnewPairs",DFreducerBis("key","MapOutput")).select(explode(col('ListnewPairs'))).select([col("col")[i] for i in range(3)])

  compteur=ReduceDF.where("col[0]=1").count()
  print(compteur)

  ##suite #fonction Reduce de CFF Dedup sur base RDD liste de chiffres pour vérifier la suppression des doublons de tuples
  DF=ReduceDF.withColumnRenamed("col[1]","key").withColumnRenamed("col[2]","value").select("key","value").distinct()

  if compteur == 0:
    print("arret")
    boucle = False

t1=time.time()
#le round de python interfère l'opérateur round de pspark sql (qui s'applique à des colonnes)
print("c'est la fin; durée totale de :",t1-t0)

8552232
4758451
3278772
3888454
1905323
86783
1318
0
arret
c'est la fin; durée totale de : 1355.478346824646


In [0]:
#zone de contrôle préparation contrôle de fonctionnement de la version bis du compteur / à supprimer à la fin
DF=DF_Init.withColumn("_c0",DF_Init["_c0"].cast("int")).withColumnRenamed("_c0","key")\
                     .withColumn("_c1",DF_Init["_c1"].cast("int")).withColumnRenamed("_c1","value")
MapDF=DF.union(DF.select('value','key')).groupBy("key").agg(collect_list("value").alias("MapOutput"))
ReduceDF_comptage=MapDF.withColumn("ListnewPairs",DFreducer("key","MapOutput"))
ReduceDF=ReduceDF_comptage.select(explode(col('ListnewPairs'))).select([col("col")[i] for i in range(2)])
ReduceDF_comptage.show(10)

In [0]:
#zone de vérification du fonctionnement du compteur pour récupérer le décompte dans une variable python / à supprimer à la fin 
compteur=ReduceDF_comptage.withColumn("minimum_array",array_min("MapOutput")).filter(col('key') > col('minimum_array'))\
.withColumn("compteur",size("MapOutput")-1).agg(sum(col('compteur'))).collect()[0][0] 
print(compteur)

171914
